In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 145kB/s  eta 0:00:01                      | 16.2MB 6.9MB/s eta 0:00:30     |███████████████████████▊        | 161.7MB 40.3MB/s eta 0:00:02ta 0:00:01�██▍| 213.7MB 38.2MB/s eta 0:00:01
     |████████████████████████████████| 204kB 38.9MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-27 03:36:58--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-27 03:36:58--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-27 03:36:58--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving 

In [23]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [24]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler,Normalizer
from pyspark.ml.linalg import Vectors

In [25]:
#preprocessing
indexer = StringIndexer(inputCol="class",outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vA = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer = Normalizer(inputCol="features",outputCol="features_norm",p=1.0)

In [26]:
df.createOrReplaceTempView('df')
df_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")


In [27]:
#for binary classification
splits = df_class.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]

In [28]:
#gradient boosting
from pyspark.ml.classification import GBTClassifier
GB = GBTClassifier(labelCol="label",featuresCol="features",maxIter=10)

In [29]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer,encoder,vA,normalizer,GB])

In [30]:

#training
model = pipeline.fit(df_train)

In [31]:
pred = model.transform(df_train)

In [32]:
#evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("label")
binEval.evaluate(pred)

0.9110346522487097

In [33]:
#testing
model = pipeline.fit(df_test)

In [34]:
pred = model.transform(df_train)

In [35]:
#evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("label")
binEval.evaluate(pred)

0.9075632833620054

In [36]:
#Hyper Parameter tunning
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder


In [37]:
#using grid Search
paramGrid = ParamGridBuilder().addGrid(normalizer.p,[1.0,2.0,10.0]).addGrid(GB.maxBins,[2,4,8,16]).addGrid(GB.maxDepth, [2,4,8,16]).build()

In [38]:
crossVal = CrossValidator(estimator= pipeline, estimatorParamMaps=paramGrid, evaluator = MulticlassClassificationEvaluator(),numFolds=4)

In [39]:
model = crossVal.fit(df_train)

In [40]:
predictions = model.transform(df_test)


In [41]:
#accuracy
binEval.evaluate(predictions)

0.9127317676143387